In [1]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import GEOparse
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
class GEOParser:
    """
    Parser for GEO datasets using GEOparse.
    First version specific for GSE121239 (Lúpus).

    Parameters
    -----------
    geo_id : str
        GEO Series ID (e.g., 'GSE121239').
    destdir : str
        Directory to download GEO data.
    gene_id_cols : list of str, optional
        Columns to use for gene ID mapping. Default is ['ID', 'Gene Symbol', 'Gene Title'].
    meta_map : dict, optional
        Mapping of metadata fields to their index in characteristics_ch1. 
    """
    def __init__(self, geo_id, destdir, gene_id_cols=None, meta_map=None):
        self.geo_id = geo_id
        self.destdir = destdir
        self.gene_id_cols = gene_id_cols or ['ID', 'Gene Symbol', 'Gene Title']
        self.meta_map = meta_map
        self.gse = None
        self.gpl = None

    def load(self):
        """
        Load GEO Series data.
        """
        self.gse = GEOparse.get_GEO(geo=self.geo_id, destdir=self.destdir, silent=True)

    def try_cast(self, val, typ):
        """
        Try to cast a value to a specified type, return NaN on failure.

        Parameters
        ----------
        val : any
            Value to cast.
        typ : type
            Target type for casting.
        """
        try:
            return typ(val)
        except (ValueError, TypeError):
            return np.nan

    def select_main_gene(self, gene_str, separator='///'):
        """
        Select the main gene from a string of gene symbols separated by a given separator.
        Parameters
        ----------
        gene_str : str
            String containing gene symbols separated by the specified separator.
        separator : str, optional
            Separator used in the gene string. Default is '///'.
        Returns
        -------
        main_gene : str or np.nan
            The selected main gene symbol, or NaN if input is NaN.
        """
        if pd.isna(gene_str):
            return np.nan
        # Lista de genes encontrados
        genes = [g.strip() for g in gene_str.split(separator)]

        # Prioriza genes que não começam com LOC (Locus genômico)
        known_genes = [g for g in genes if not g.startswith('LOC')]
        if known_genes:
            return known_genes[0]
        return genes[0]  # Se só tem LOC, retorna o primeiro

    def parse(self):
        """
        Parse the loaded GEO Series data.

        Returns
        -------
        gene_data: pd.DataFrame
            Dataframe with Gene expression data.
        patient_data: pd.DataFrame
            Dataframe with Patient metadata.
        """
        if self.gse is None:
            self.load()

        # 1. Cria listas com metadados e expressao dos paciente e expressao genica 
        patient_meta_list = []
        gene_data_list = []

        # 2. Itera sobre os GSMs para extrair metadados e dados de expressão
        for gsm_name, gsm in self.gse.gsms.items():

            # 2.1. Extrai metadados do paciente
            meta = gsm.metadata['characteristics_ch1']
            patient_meta = {'sample_code': gsm_name, 'treatment_protocol': gsm.metadata['treatment_protocol_ch1'][0]}

            # 2.2. Parse de metadados de acordo com o mapeamento fornecido
            for key, idx in self.meta_map.items():
                # 2.3 . Extrai valor da lista de metadados e faz cast apropriado
                val = meta[idx].split(':')[1].strip() if len(meta) > idx else None
                if key == 'sledai':
                    val = self.try_cast(val, int)
                elif key == 'visit_date':
                    val = self.try_cast(val, pd.Timestamp)
                elif key == 'neutrophil_percent':
                    val = self.try_cast(val, float)
                # 2.4. Adiciona informação ao dicionario de metadados do paciente
                patient_meta[key] = val
            
            patient_meta_list.append(patient_meta)

            # 2.5. Extrai dados de expressão gênica
            gene_df = gsm.table[['ID_REF', 'VALUE']].copy()
            gene_df['sample_code'] = gsm_name
            gene_data_list.append(gene_df)

        # 3. Concatena dados de expressão gênica em um único DataFrame
        gene_data = pd.concat(gene_data_list, ignore_index=True)

        # 4. GPLS: plataforma com anotação de genes, incluindo nome e símbolo
        self.gpl = self.gse.gpls[list(self.gse.gpls.keys())[0]].table

        # 5. Recupera nome e título do gene com base no ID
        gene_data = gene_data.merge(self.gpl[self.gene_id_cols], left_on='ID_REF', right_on='ID', how='left')
        gene_data = gene_data.rename(columns={'VALUE': 'gene_expression_value', 'Gene Symbol': 'gene_symbol', 'Gene Title': 'gene_title'})
        gene_data = gene_data[['sample_code', 'ID', 'gene_symbol', 'gene_title', 'gene_expression_value']]

        # 6. Metadados do paciente em dataframe
        patient_data = pd.DataFrame(patient_meta_list)
        patient_data['patient_id'] = patient_data['patient_id'].replace({'NA': np.nan})
        patient_data = patient_data.sort_values(by=['patient_id', 'visit_date'])

        return gene_data, patient_data

### Loading Data

In [3]:
meta_map = {
    'state': 0,
    'patient_id': 1,
    'sledai': 2,
    'visit_date': 3,
    'neutrophil_percent': 5
}

# Cria objeto parser
parser = GEOParser(geo_id="GSE121239", destdir="../../data/raw/", meta_map=meta_map)

# Parse dos dados e armazena em tabelas
gene_data, patient_data = parser.parse()

# Mantem o "gene principal" em casos de genes separados por '///'
gene_data['main_gene'] = gene_data['gene_symbol'].apply(parser.select_main_gene)

# Remove valores de microarray (Affymetrix) -> Controle de qualidade de experimento
gene_data = gene_data[~gene_data['ID'].str.startswith('AFFX')]

/home/cs331285/anaconda3/envs/ml/lib/python3.13/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")


In [4]:
patient_data = patient_data.sort_values(by=['patient_id', 'visit_date'])
patient_data['sledai_variation'] = patient_data.groupby('patient_id')['sledai'].diff()
patient_data['visit_date_diff'] = patient_data.groupby('patient_id')['visit_date'].diff()

# 70/227 (31%) tiveram aumento de SLEDAI entre visitas
# 157/227 (69%) tiveram diminuição ou estabilidade de SLEDAI entre visitas
# Não existe padrão de tempo entre visitas (provavelmente não era programada)

In [5]:
patient_data.head()

,sample_code,treatment_protocol,state,patient_id,sledai,visit_date,neutrophil_percent,sledai_variation,visit_date_diff
105,GSM3428415,Standard.,Systemic Lupus Erythematosus,1001,4,2009-10-08,65.2,NaN,NaT
106,GSM3428416,Standard.,Systemic Lupus Erythematosus,1001,2,2010-01-11,61.5,-2.0,95 days
107,GSM3428417,Standard.,Systemic Lupus Erythematosus,1001,6,2010-03-29,57.5,4.0,77 days
108,GSM3428418,Standard.,Systemic Lupus Erythematosus,1041,2,2009-09-24,73.8,NaN,NaT
109,GSM3428419,Standard.,Systemic Lupus Erythematosus,1041,10,2009-12-10,64.1,8.0,77 days


In [5]:
gene_data.head()

,sample_code,ID,gene_symbol,gene_title,gene_expression_value,main_gene
0,GSM3428310,1007_PM_s_at,DDR1,discoidin domain receptor tyrosine kinase 1,4.955257,DDR1
1,GSM3428310,1053_PM_at,RFC2,"replication factor C (activator 1) 2, 40kDa",5.984784,RFC2
2,GSM3428310,117_PM_at,HSPA6,heat shock 70kDa protein 6 (HSP70B'),9.477945,HSPA6
3,GSM3428310,121_PM_at,PAX8,paired box 8,4.553229,PAX8
4,GSM3428310,1255_PM_g_at,GUCA1A,guanylate cyclase activator 1A (retina),1.921190,GUCA1A


In [ ]:
sledai_counts = patient_data.query("sledai > 0")['sledai'].value_counts().sort_index()

fig = go.Figure(
    data=[
        go.Bar(
            x=sledai_counts.index,
            y=sledai_counts.values,
            marker=dict(
                color="#39b3c2",
                line=dict(color='rgba(44,44,84,1)', width=1.5)
            ),
            width=0.7,
            hovertemplate='SLEDAI: %{x}<br>Frequência: %{y}<extra></extra>'
        )
    ]
)

fig.update_layout(
    title=dict(x=0.5, text='Distribuição dos valores de SLEDAI'),
    xaxis=dict(title='SLEDAI', dtick=1),
    yaxis_title='Frequência',
    template='plotly_white',
    bargap=0.15,
    width=700,
    height=400,
)

fig.show()

### Classificando lesões

In [6]:
def classify_lesion(sledai_score):
    if sledai_score == 0:
        return 'healthy'
    if 1 <= sledai_score <= 6:
        return 'leve'
    elif 7 <= sledai_score <= 11:
        return 'media'
    else:
        return 'grave'

def classify_lesion_more_granular(sledai_score):
    if sledai_score == 0:
        return 'healthy'
    if 1 <= sledai_score <= 3:
        return 'levissimo'
    if 4 <= sledai_score <= 6:
        return 'leve'
    elif 7 <= sledai_score <= 9:
        return 'media'
    else:
        return 'gravissimo'

In [7]:
# Grupos seguindo a separação padrão
patient_data['class'] = patient_data['sledai'].apply(classify_lesion)
patient_data['class_group'] = np.where(patient_data['class'].isin(['grave', 'media']), 'medio/grave', 'leve')
patient_data['class_group'] = np.where(patient_data['class'] == 'healthy', 'healthy', patient_data['class_group'])

# Aumentando o número de grupos
patient_data['class_granular'] = patient_data['sledai'].apply(classify_lesion_more_granular)

In [261]:
# gene_data.to_parquet('../../data/processed/gene_data.parquet', index=False)
# patient_data.to_csv('../../data/processed/patient_data.csv', index=False)

In [10]:
def plot_pie_distribution(patient_data, col='class'):
    if col == 'class':
        colors = ["#ABACB2", "#bb77ba", "#39b3c2", "#7e7ae7"]
    elif col == 'class_group':
        colors = ["#39b3c2", "#7e7ae7"]
    else:
        colors = ["#7e7ae7", "#39b3c2"]
        
    fig = go.Figure(data=[go.Pie(
        labels=[label.capitalize() for label in np.sort(patient_data[col].unique())],
        values=patient_data[col].value_counts(normalize=True).sort_index().values,
        hole=0.34,
        marker_colors=colors,
        textinfo='percent+label',
        insidetextorientation='radial',
        textfont=dict(family='Arial Black')
    )])
    
    fig.update_layout(
        title=dict(x=0.5, text='Distribuição do Grau de Lesão (SLEDAI) nos Pacientes'),
        template='plotly_white',
        showlegend=False,
        width=500
    )
    
    fig.show()

fig = plot_pie_distribution(patient_data, col='class')
fig = plot_pie_distribution(patient_data.query("class_group != 'healthy'"), col='class_group')

In [ ]:
# Filtra a observação com maior SLEDAI por paciente
patient_data['max_sledai'] = patient_data.groupby('patient_id')['sledai'].transform('max')
patient_data_max_sledai = patient_data.query("sledai == max_sledai").drop(columns=['max_sledai'])
patient_data_max_sledai = patient_data_max_sledai.groupby('patient_id').first().reset_index()

In [279]:
patient_data_max_sledai.class_group.value_counts()

class_group
leve           39
medio/grave    26
Name: count, dtype: int64

## Expressão Diferencial Leve vs. Medio/Grave

In [ ]:
df = gene_data.merge(patient_data_max_sledai, on='sample_code')
df = df.sort_values(by=['patient_id', 'sledai'])

In [ ]:
import scipy.stats as stats
import concurrent.futures
import os
import pickle

# Lista de genes
genes = df['main_gene'].dropna().unique()

# Carrega arquivo com genes já processados
if os.path.exists('ttest_gene_expression.pkl'):
    with open('ttest_gene_expression.pkl', "rb") as f:
        checkpoint = pickle.load(f)
else:
    checkpoint = {}

def ttest_for_gene(gene):
    if gene in checkpoint:
        return checkpoint
    
    # Separa gene dos grupos de análise
    group1 = df[(df['main_gene'] == gene) & (df['class_group'] == 'leve')]['gene_expression_value']
    group2 = df[(df['main_gene'] == gene) & (df['class_group'] == 'medio/grave')]['gene_expression_value']

    # Gera pvalor da expressão diferencial
    _, pval = stats.ttest_ind(group1, group2, equal_var=False)
    checkpoint[gene] = {'pval': pval, 'mean_leve': group1.mean(), 'mean_grave': group2.mean()}
    # checkpoint.update({'gene': gene, 'pval': pval, 'mean_leve': group1.mean(), 'mean_grave': group2.mean()})

    # Atualiza arquivo de checkpoint
    with open('ttest_gene_expression.pkl', "wb") as f:
        pickle.dump(checkpoint, f)

    return checkpoint

with concurrent.futures.ProcessPoolExecutor(max_workers=4, mp_context=None) as executor:
    results = list(executor.map(ttest_for_gene, genes))


In [ ]:
# DataFrame com resultados e log2FC
res_df = pd.DataFrame(results)
res_df['log2fc'] = np.log2(res_df['mean_grave'] + 1) - np.log2(res_df['mean_leve'] + 1)
res_df['log2fc_div'] = np.log2(res_df['mean_grave'] / res_df['mean_leve'])
res_df = res_df.sort_values('pval')
res_df.to_csv('../../data/processed/leve_grave_gene_expression.csv', index=False)

# Genes mais diferenciais
top_genes = res_df[res_df['pval'] < 0.05].sort_values('log2fc', ascending=False)
top_genes[['gene']].to_csv('../../data/processed/top_differential_genes_leve_grave.csv', index=False)

In [ ]:
import plotly.graph_objects as go

# Adiciona coluna para -log10(pval)
res_df['-log10_pval'] = -np.log10(res_df['pval'])

# Define cores: destaque genes significativos
colors = np.where((res_df['pval'] < 0.05) & (abs(res_df['log2fc']) > 1), '#A259C6', '#4CB944')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=res_df['log2fc'],
    y=res_df['-log10_pval'],
    mode='markers',
    marker=dict(
        color=colors,
        size=8,
        line=dict(width=0.5, color='DarkSlateGrey')
    ),
    text=res_df['gene'],
    hovertemplate='Gene: %{text}<br>log2FC: %{x:.2f}<br>-log10(pval): %{y:.2f}<extra></extra>'
))

fig.update_layout(
    title='Volcano Plot - Expressão Diferencial',
    xaxis_title='log2 Fold Change',
    yaxis_title='-log10(p-valor)',
    template='plotly_white',
    width=700,
    height=500
)

fig.show()

## Expressão Diferencial Levissimo vs. Gravissimo (Mesmos pacientes com os dois casos)

In [8]:
aux = patient_data.groupby('patient_id').class_granular.unique().reset_index()
patient_ids = []
for id in aux.patient_id:
    classes = aux[aux['patient_id'] == id]['class_granular'].values[0]
    if 'levissimo' in classes and 'gravissimo' in classes:
        patient_ids.append(id)

print("Pacientes com Levissimo e Gravissimo:", len(patient_ids))

Pacientes com Levissimo e Gravissimo: 11


In [11]:
patient_leve_grave = patient_data.query("patient_id.isin(@patient_ids) and class_granular.isin(['levissimo', 'gravissimo'])").sort_values(by=['patient_id', 'sledai'])
patient_leve_grave = pd.concat([patient_leve_grave.groupby('patient_id').first().reset_index(), patient_leve_grave.groupby('patient_id').last().reset_index()], ignore_index=True).sort_values(by=['patient_id', 'sledai'])

plot_pie_distribution(patient_leve_grave, col='class_granular')

In [16]:
df_levissimo_gravissimo = gene_data.merge(patient_leve_grave, on='sample_code')

In [17]:
import scipy.stats as stats
import concurrent.futures
import os
import pickle

# Lista de genes
genes_test = [
    "RPL4",
    "RPLP0",
    "EEF2",
    "RPL12",
    "KIAA0649", # "PPP1R26",
    "EEF1G",
    "RPS17",
    "KARS"#"KARS1"
]

checkpoint = {}

def ttest_paired_for_gene(gene):
    # Separa gene dos grupos de análise
    group1 = df_levissimo_gravissimo[(df_levissimo_gravissimo['main_gene'] == gene) & (df_levissimo_gravissimo['class_granular'] == 'levissimo')]['gene_expression_value']
    group2 = df_levissimo_gravissimo[(df_levissimo_gravissimo['main_gene'] == gene) & (df_levissimo_gravissimo['class_granular'] == 'gravissimo')]['gene_expression_value']

    # Gera pvalor da expressão diferencial para o mesmo paciente 
    _, pval = stats.ttest_rel(group1, group2)
    checkpoint.update({'gene': gene, 'pval': pval, 'mean_leve': group1.mean(), 'mean_grave': group2.mean()})

    return checkpoint

with concurrent.futures.ProcessPoolExecutor(max_workers=4, mp_context=None) as executor:
    results = list(executor.map(ttest_paired_for_gene, genes_test))

In [19]:
# DataFrame com resultados e log2FC
res_df_2 = pd.DataFrame(results)
res_df_2['log2fc'] = np.log2(res_df_2['mean_grave'] / res_df_2['mean_leve'])
res_df_2 = res_df_2.sort_values('pval')

In [20]:
res_df_2

,gene,pval,mean_leve,mean_grave,log2fc
1,RPLP0,0.000131,10.384573,10.153425,-0.032475
3,RPL12,0.000765,11.465301,11.234577,-0.029329
5,EEF1G,0.042925,11.347142,11.213275,-0.017121
7,KARS,0.046986,8.982409,8.776657,-0.033431
2,EEF2,0.188004,10.338773,10.218535,-0.016877
4,KIAA0649,0.228485,5.852116,5.673915,-0.044614
6,RPS17,0.254481,11.387129,11.304829,-0.010465
0,RPL4,0.834899,11.549234,11.535096,-0.001767


## Analise dos Genes de referencia no artigo para analise de p-valor

In [ ]:
genes_artigo = [
    "CEACAM6",
    "UCHL1",
    "ARFGEF3",
    "AMPH",
    "SERPINB10",
    "TACSTD2",
    "OTX1",
    "SORBS2",
    "TRIM64B",
    "SORCS3",
    "DRAXIN",
    "PCDHGA10"
]

In [105]:
res_df.query("gene in @genes_artigo").sort_values(by='pval')

,gene,pval,mean_leve,mean_grave,log2fc,log2fc_div
7634,AMPH,0.081849,2.942120,3.107397,0.059253,0.078850
19735,OTX1,0.117664,1.940956,1.849456,-0.045599,-0.069667
12135,SERPINB10,0.126287,2.023613,2.233789,0.096952,0.142559
2011,UCHL1,0.466686,2.608853,2.731782,0.048324,0.066427
12424,SORCS3,0.512043,2.833607,2.769936,-0.024162,-0.032787
6588,CEACAM6,0.605262,4.261841,4.371750,0.029824,0.036734
1844,SORBS2,0.870385,2.513468,2.501145,-0.005069,-0.007091
11233,PCDHGA10,0.943041,1.785047,1.779577,-0.002836,-0.004428
5665,TACSTD2,0.983207,2.838732,2.841243,0.000943,0.001275


### Correlação SLEDAI vs Neutrófilos

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    x=patient_data['sledai'],
    y=patient_data['neutrophil_percent'],
    # mode='markers',
    # marker=dict(color='indianred', size=8, opacity=0.6),
    # name='Pacientes'
))
fig.show()

### Analisando Genes não encontrados pelo STRING

In [ ]:
# Leitura do mapping de genes do STRING
f = open('../../data/external/string_found_genes_mapping.tsv')
content = f.read()
f.close()

# Transformando string em lista
content = content.split('\n')[1:]

# Lista de genes encontrados no STRING
matched_genes = [line.split('\t')[1] for line in content if line]

In [63]:
# Lista dos top genes diferenciais
all_top_genes = pd.read_csv('../../data/processed/top_differential_genes_leve_grave.csv').gene.tolist()

# Lista de genes não encontrados no STRING
unmatched_genes = list(set(all_top_genes) - set(matched_genes))

In [ ]:
# Recuperando gene original dos genes não encontrados do tipo "LOC"
gene_data.query("main_gene.isin(@unmatched_genes) and gene_symbol.str.contains('LOC')").gene_symbol.unique()

array(['LOC142937', 'LOC100288109 /// LOC203274', 'LOC150051',
       'LOC441208', 'LOC283761', 'LOC348751', 'LOC285389', 'LOC440993',
       'LOC147670', 'LOC100131848', 'LOC100133116',
       'LOC100132864 /// LOC100293492 /// LOC389906 /// LOC441528 /// LOC729162',
       'LOC148189', 'LOC100129510', 'LOC283856', 'LOC283027', 'LOC644090',
       'LOC100289061', 'LOC92973',
       'LOC388152 /// LOC727751 /// LOC727849 /// LOC80154',
       'LOC100287132', 'LOC729580', 'LOC100132891', 'LOC100292443',
       'LOC100288617', 'LOC100289209', 'LOC100288431', 'LOC149837',
       'LOC100133985', 'LOC100130938', 'LOC441052', 'LOC153684',
       'LOC100268168', 'LOC100288675', 'LOC344595', 'LOC100287428',
       'LOC100288507', 'LOC401097', 'LOC100288722', 'LOC441046',
       'LOC100287290', 'LOC100130429', 'LOC646548', 'LOC642980',
       'LOC646778', 'LOC100287621'], dtype=object)

In [ ]:
# Recuperando gene original dos genes não encontrados diferentes de "LOC"
gene_data.query("main_gene.isin(@unmatched_genes) and not main_gene.str.contains('LOC', na=False)").gene_symbol.unique()

array(['C10orf25', 'CSN1S2A', 'C17orf77', 'MGC20647', 'BAGE', 'TPTE2P2',
       'SNORA65', 'FLJ35024', 'BCL8', 'KIAA1908', 'FLJ34521', 'FLJ43944',
       'PP13439', 'DLEU1', 'HCP5', 'RPS14P3', 'HLA-DRB4', 'HCG26',
       'SNORA21', 'TRPC2', 'ZNF767', 'FLJ20712', 'C21orf96', 'C17orf86',
       'DDX11L2', 'SNHG11', 'SDHAP1', 'DSCR9', 'SNORD114-3', 'FLJ12334',
       'ANKRD19', 'RRN3P3', 'FLJ45513', 'SNORA28'], dtype=object)

In [ ]:
# Visualizacoes uteis

# gse = GEOparse.get_GEO(geo="GSE121239", destdir="../../data/raw/", silent=True)
# gse.gsms['GSM3428621'].metadata # Metadados pacientes
# gse.gsms['GSM3428621'].table # Tabela de genes
# gpl.head() # De para de Genes

{'title': ['PBMC_SLE2132_v4'],
 'geo_accession': ['GSM3428621'],
 'status': ['Public on Oct 15 2018'],
 'submission_date': ['Oct 15 2018'],
 'last_update_date': ['Oct 15 2018'],
 'type': ['RNA'],
 'channel_count': ['1'],
 'source_name_ch1': ['PBMC_SLE2132_v4'],
 'organism_ch1': ['Homo sapiens'],
 'taxid_ch1': ['9606'],
 'characteristics_ch1': ['disease state: Systemic Lupus Erythematosus',
  'patient id: 2132',
  'sledai: 2',
  'visit date: 2010-12-15',
  'tissue: PBMC',
  'imputed neutrophil percentage: 56.2'],
 'treatment_protocol_ch1': ['Standard.'],
 'growth_protocol_ch1': ['Whole Blood samples in collected fom PAXgene tubes.'],
 'molecule_ch1': ['total RNA'],
 'extract_protocol_ch1': ['The RNA was extracted with Trizol reagent (Life Technologies) and cleaned up with RNA easy (Qiagen) according to the manufacturer’s protocols.'],
 'label_ch1': ['Biotin'],
 'label_protocol_ch1': ['NuGEN Ovation.'],
 'hyb_protocol': ['Standard Affymetrix protcol.'],
 'scan_protocol': ['HTAPS scan was